### *IPCC SR15 scenario assessment*

<img style="float: right; height: 100px; margin-top: 10px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Analysis of the primary energy development <br /> in illustrative pathways

This notebook computes indicators and diagnostics of the primary-energy timeseries by fuel in **Figure 2.15**, 
focusing on the illustrative pathways and the IEA's 'Faster Transition Scenario' using the 'World Energy Model' 
in the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
marker = specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
cats.remove('Above 2C')

In [ ]:
years = [2030, 2050, 2100]

In [ ]:
df = sr1p5.filter(category=cats, year=years)

## Set specifications for filter and plotting

In [ ]:
save_name = 'output/fig2.15{}.png'

In [ ]:
filter_args = dict(df=sr1p5, category=cats, marker=None, join_meta=True)

## Retrieve historical data from IEA Energy Statistics and rename variables

In [ ]:
hist = sr1p5.filter(model='Reference', scenario='IEA Energy Statistics (r2017)')

In [ ]:
fossil_vars = [
    'Primary Energy|Coal',
    'Primary Energy|Oil',
    'Primary Energy|Gas'
]

ren_vars = [
    'Primary Energy|Geothermal',
    'Primary Energy|Hydro',
    'Primary Energy|Ocean',
]

hist_mapping = {}

for f in fossil_vars:
    hist_mapping.update({f: 'Fossil without CCS'})
for r in ren_vars:
    hist_mapping.update({r: 'Other renewables'})

hist_mapping.update({'Primary Energy|Biomass': 'Biomass without CCS'})

hist.rename({'variable': hist_mapping}, inplace=True)

In [ ]:
hist_args = dict(color='black', linestyle='dashed', linewidth=1)

## Add IEA's 'Faster Transition Scenario' to the set of marker scenarios for comparison

In [ ]:
m = 'IEA WEM'
col = 'marker'
df.set_meta(m, col,
            df.filter(model='IEA World Energy Model 2017',
                      scenario='Faster Transition Scenario'))
rc.update({'marker': {col: {m: 'o'}},
           'c': {col: {m: 'red'}},
           'edgecolors': {col: {m: 'black'}}}
         )
marker += [m]

## Rename variables for plots

In [ ]:
variable_mapping = [
    ('Fossil without CCS', 'Primary Energy|Fossil|w/o CCS', 'black'),
    ('Fossil with CCS', 'Primary Energy|Fossil|w/ CCS', 'grey'),
    ('Biomass without CCS',
         ['Primary Energy|Biomass|Modern|w/o CCS',
          'Primary Energy|Biomass|Traditional'], 'forestgreen'),
    ('Biomass with CCS', 'Primary Energy|Biomass|Modern|w/ CCS', 'limegreen'),
    ('Nuclear', 'Primary Energy|Nuclear', 'firebrick'),
    ('Wind', 'Primary Energy|Wind', 'lightskyblue'),
    ('Solar', 'Primary Energy|Solar', 'gold'),
    ('Other renewables',
         ['Primary Energy|Ocean',
          'Primary Energy|Geothermal',
          'Primary Energy|Hydro'], 'darkorange'),
]

In [ ]:
variables = []
mapping = {}

for (name, variable, color) in variable_mapping:
    variables.append(name)
    if isinstance(variable, list):
        for v in variable:
            mapping.update({v: name})
    else:
        mapping.update({variable: name})
    rc.update({'color': {'marker': {name: color}}})

df.rename({'variable': mapping}, inplace=True)
hist.rename({'variable': mapping}, inplace=True)

The "LowEnergyDemand" scenario by design does not have CCS, and does not explicitly include these variables. Therefore, the variable mapping is implemented differently.

In [ ]:
led_mapping = {
    'Primary Energy|Biomass': 'Biomass without CCS',
    'Primary Energy|Coal': 'Fossil without CCS',
    'Primary Energy|Gas': 'Fossil without CCS',
    'Primary Energy|Oil': 'Fossil without CCS',
}

df.data = df.data.append(
    df.filter(scenario='LowEnergyDemand',
              variable=led_mapping)
    .rename({'variable': led_mapping})
    .data
)

In [ ]:
df.filter(variable=variables, inplace=True)
hist.filter(variable=variables + ['Primary Energy'],
            inplace=True)

## Plot energy system development by marker scenario

In [ ]:
w, h = plt.figaspect(0.30)
fig = plt.figure(figsize=(w, h))
ymax = 1150
hist_yr = 2015

_years = len(years) - 1
label_list = []

w = 0.5 / len(years)

for i, m in enumerate(marker):
    _df = df.filter(marker=m).timeseries()
    meta = _df.iloc[0].name[0:2]
    _label = '{}\n{}\n({})'.format(meta[0], meta[1], m)
    _df.index = _df.index.droplevel([0, 1, 2, 4])
    
    # use _df.columns because not all scenarios extend until 2100
    pos = [0.5 / _years * (j - (len(_df.columns) - 1) / 2) + i
           for j in range(len(_df.columns))]
    b = [0] * len(_df.columns)

    for v in variables:
        if v in _df.index:
            lst = _df.loc[v]
            plt.bar(x=pos, height=lst, bottom=b, width=w,
                    color=rc['color']['marker'][v],
                    edgecolor='black', label=None)
            b += _df.loc[v]

    val = (
        hist.filter(variable='Primary Energy')
        .timeseries()[hist_yr]
    )
    plt.hlines(y=val, xmin=(-.4 + i),
               xmax=(.4 + i), **hist_args,
               label=None)
            
    label_list.append(_label)

    # add years at the top
    for j, yr in enumerate(_df.columns):
        plt.text(pos[j] - 0.1, ymax * 1.05, yr)    
    
# add legend entries
plt.hlines(y=[], xmin=[], xmax=[], **hist_args,
           label='{} Primary Energy (IEA Energy Statistics 2017)'.format(hist_yr))
for v in variables:
    plt.scatter(x=[], y=[], color=rc['color']['marker'][v], label=v)

plt.legend()
plt.grid(False)
plt.xlim(-0.6, (i + 0.6))
plt.xticks(range(0, i + 1), label_list)
plt.vlines(x=[_i + 0.5 for _i in range(i)], ymin=0, ymax=ymax, colors='white')
plt.ylim(0, ymax)
plt.ylabel('Primary energy by illustrative pathway (EJ/y)')

fig.savefig(save_name.format('a_primary_energy_by_marker'))

## Plot energy system development by fuel for all 1.5°C and 2°C pathways

In [ ]:
w, h = plt.figaspect(0.3)
fig = plt.figure(figsize=(w, h))
ymax = 550
hist_yr = 2015

_years = len(years) - 1
label_list = []

def marker_args(m):
    return dict(zorder=4,
                edgecolors=rc['edgecolors']['marker'][m],
                c=rc['c']['marker'][m],
                marker=rc['marker']['marker'][m],
                linewidths=1)

for i, v in enumerate(variables):
    _df = df.filter(variable=v).timeseries()
    _df.index = _df.index.droplevel([2, 3, 4])
    _df = pyam.filter_by_meta(_df, df, marker=None, join_meta=True)
    
    for j, y in enumerate(years):
        lst = _df[y][~np.isnan(_df[y])]
        pos = 0.5 / _years * (j - _years / 2) + i
        
        outliers = len(lst[lst > ymax])
        if outliers > 0:
            plt.text(pos - 0.01 * len(years), ymax * 1.01, outliers)

        p = plt.boxplot(lst, positions=[pos], 
                        whis='range',
                        patch_artist=True)
        plt.setp(p['boxes'], color=rc['color']['marker'][v])
        plt.setp(p['medians'], color='black')

        for m in marker:
            val = _df.loc[_df.marker == m, y]
            if not val.empty:
                plt.scatter(x=pos, y=val, **marker_args(m),
                            s=40, label=None)

    if v in list(hist.variables()):
        val = hist.filter(variable=v).timeseries()[hist_yr]
        plt.hlines(y=val, xmin=(-.4 + i), xmax=(.4 + i), **hist_args,
                   label=None)

    label_list.append(v)
    
# add legend entries
plt.hlines(y=[], xmin=[], xmax=[], **hist_args,
           label='{} Primary Energy (IEA Energy Statistics 2017)'.format(hist_yr))
for m in marker:
    meta = df.filter(marker=m).timeseries().iloc[0].name[0:2]
    _label = '{}|{} ({})'.format(meta[0], meta[1], m)
    plt.scatter(x=[], y=[], **marker_args(m), s=60, label=_label)

# add years at the top
for _i in range(0, i + 1):
    for j, yr in enumerate(years):
        plt.text(0.5 / _years * (j - _years / 2) + _i - 0.1,
                 ymax * 1.05, yr)

#plt.legend()
plt.grid(False)
plt.xlim(-0.6, (i + 0.6))
plt.xticks(range(0, i + 1), label_list)
plt.vlines(x=[_i + 0.5 for _i in range(i)], ymin=0, ymax=ymax, colors='white')
plt.ylim(0, ymax)
plt.ylabel('Primary energy by fuel type (EJ/y)')

fig.savefig(save_name.format('b_primary_energy_by_fuel'))